In [1]:
import numpy as np
import torch
import copy
from metabci.brainda.algorithms.deep_learning import ShallowNet_Sparse
from metabci.brainda.algorithms.utils.model_selection import (
    set_random_seeds,
    generate_kfold_indices, match_kfold_indices)
from metabci.brainda.datasets import Wang2016, BETA
# 三个SSVEP Datasets：Nakanishi2015、Wang2016、BETA
from metabci.brainda.paradigms import SSVEP
from skorch.helper import predefined_split

In [2]:
#**************************************************
# BETA数据集读取处理
#**************************************************

BETA_dataset = BETA()
subject_list = list(range(1, 71))  # 被试编号从1到70
for s in subject_list:
    BETA_dataset.data_path(subject=s, path="E:\\MetaBCI-master\\mne_data")  # 依次为每个被试设置路径
BETA_subjects_2S = list(range(1, 16))  # 使用S1–S15被试进行预训练，2s窗
BETA_subjects_3S = list(range(16, 36))  # 使用S16–S70被试进行预训练，3s窗
events = BETA_dataset.events.keys()
freq_list = [str(BETA_dataset.get_freq(event)) for event in events]  # 获得所有刺激的频率
BETA_freq_map = {i: freq for i, freq in enumerate(freq_list)}  # 标签到频率的映射
print(freq_list) # 输出频率显示

# BETA数据集论文在 2.6 Data Preprocessing 章节有以下说明
# 对于S1–S15的被试，时间窗为 2秒
# 对于S16–S70的被试，时间窗为 3秒，试次总长度为 4秒
paradigm_2s = SSVEP(
    # In order to validate the data quality by visual inspection,
    # nine parietal and occipital channels(Pz,PO3,PO5,PO4,PO6,POz,O1,Oz,and O2) were selected。
    channels=['POZ', 'PZ', 'PO3', 'PO5', 'PO4', 'PO6', 'O1', 'OZ', 'O2'],  # 选择电极通道
    intervals=[(0.14, 1.14)],  # 分析时间窗最长-0.5-2.5s ,每个试次包含事件开始前0.5秒的数据，以及事件时间窗后0.5秒的数据。对于S1–S15的被试，时间窗为 2秒
    events=freq_list,  # 选择所有刺激频率
    srate=250  # 采样率
)
paradigm_3s = SSVEP(
    # In order to validate the data quality by visual inspection,
    # nine parietal and occipital channels(Pz, PO3, PO5, PO4, PO6, POz, O1, Oz,and O2) were selected。
    channels=['POZ', 'PZ', 'PO3', 'PO5', 'PO4', 'PO6', 'O1', 'OZ', 'O2'],  # 选择电极通道
    intervals=[(0.14, 1.14)],  # 分析时间窗最长-0.5-3.5s ,每个试次包含事件开始前0.5秒的数据，以及事件时间窗后0.5秒的数据。对于S16–S70的被试，时间窗为 3秒，试次总长度为 4秒
    events=freq_list,  # 选择所有刺激频率
    srate=250  # 采样率
)

# BETA数据集，已经通过matlab的eegfit，进行了3-90HZ的带通滤波，故此处不再进行滤波处理
# BETA_wof数据集没有进行滤波处理，链接已经
# 在tsinghua.py把BETA_URL做以下修改即可使用未滤波的版本
# BETA_URL = "https://bci.med.tsinghua.edu.cn/upload/liubingchuan_BETA_wof/"


# 数据存储为一个四维张量 [channel, time point, block, condition]
#                    [   64,       750,      4,      40    ]


# 获取训练数据
X_2s, y_2s, meta_2s = paradigm_2s.get_data(
    BETA_dataset,
    subjects=BETA_subjects_2S,
    return_concat=True,
    n_jobs=None,
    verbose=False
)

# 获取训练数据
X_3s, y_3s, meta_3s = paradigm_3s.get_data(
    BETA_dataset,
    subjects=BETA_subjects_3S,
    return_concat=True,
    n_jobs=None,
    verbose=False
)

# 选择使用的BETA数据
# 2s和3s的数据shape不同，不能直接拼接，是否填0补充相同大小拼接？
X_BETA = X_3s
y_BETA = y_3s
meta_BETA = meta_3s
print(X_BETA.shape)
print(meta_BETA)


--------ssssss, /upload/liubingchuan/S1-S10.tar.gz
--------ssssss, /upload/liubingchuan/S1-S10.tar.gz
--------ssssss, /upload/liubingchuan/S1-S10.tar.gz
--------ssssss, /upload/liubingchuan/S1-S10.tar.gz
--------ssssss, /upload/liubingchuan/S1-S10.tar.gz
--------ssssss, /upload/liubingchuan/S1-S10.tar.gz
--------ssssss, /upload/liubingchuan/S1-S10.tar.gz
--------ssssss, /upload/liubingchuan/S1-S10.tar.gz
--------ssssss, /upload/liubingchuan/S1-S10.tar.gz
--------ssssss, /upload/liubingchuan/S1-S10.tar.gz
--------ssssss, /upload/liubingchuan/S11-S20.tar.gz
--------ssssss, /upload/liubingchuan/S11-S20.tar.gz
--------ssssss, /upload/liubingchuan/S11-S20.tar.gz
--------ssssss, /upload/liubingchuan/S11-S20.tar.gz
--------ssssss, /upload/liubingchuan/S11-S20.tar.gz
--------ssssss, /upload/liubingchuan/S11-S20.tar.gz
--------ssssss, /upload/liubingchuan/S11-S20.tar.gz
--------ssssss, /upload/liubingchuan/S11-S20.tar.gz
--------ssssss, /upload/liubingchuan/S11-S20.tar.gz
--------ssssss, /uploa

In [3]:
#**************************************************
# BETA数据集  S16-S31 
#所有折准确率: [0.31375, 0.3275, 0.36875, 0.40875]
#平均准确率： 0.3546875
#**************************************************

# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# 设置随机种子
set_random_seeds(42)
kfold = 4
print(X_BETA.shape)  # (8800, 9, 750)  8800= 40刺激 x 4个试次 x 55人
indices = generate_kfold_indices(meta_BETA, kfold=kfold)

# 预训练阶段
n_channels = X_BETA.shape[1] # 输入信号的通道数
n_samples = X_BETA.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
shared_ratio = 0.9
alpha = 1
epsilon = 30
n_time_filters = 20
time_kernel = 25
n_space_filters = 15
pool_kernel = 8
pool_stride = 4

BETA_save_path = "BETA_shallownet_mask_1s_0.9.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = ShallowNet_Sparse(n_time_filters, time_kernel,n_space_filters,pool_kernel, pool_stride, n_channels, n_samples, n_classes, shared_ratio, alpha, epsilon)
    estimator.set_params(optimizer__lr=1e-2,device=device)  #  optimizer__lr=3e-3,默认1e-2
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_BETA, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_BETA[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_BETA[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_BETA[new_train_ind], y_BETA[new_train_ind])
    # 计算验证集准确率
    val_preds = estimator.predict(X_BETA[new_valid_ind])
    val_acc = np.mean(val_preds == y_BETA[new_valid_ind])
    #print("当前折验证集准确率：", val_acc)
    # 测试集评估
    p_labels = estimator.predict(X_BETA[new_test_ind])
    p_freqs = [BETA_freq_map [label] for label in p_labels]
    y_freqs = [BETA_freq_map [label] for label in y_BETA[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_BETA[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    print(f"所有折准确率: {pre_accs}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, BETA_save_path)


cuda
(3200, 9, 250)
in:825
out:40
train_ind数量: 2400, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0288        6.6839       0.0362        4.9669     +  0.0100  0.9446
      2       0.0362        5.6438       0.0362        4.8335     +  0.0100  0.6014
      3       0.0617        5.1400       0.0250        4.5943     +  0.0100  0.5883
      4       0.0929        4.5927       0.0675        4.5468     +  0.0100  0.5631
      5       0.1542        4.0703       0.1000        4.2612     +  0.0100  0.5693
      6       0.2196        3.6318       0.1787        3.7465     +  0.0100  0.5790
      7       0.2975        3.0419       0.2175        3.5196     +  0.0100  0.5981
      8       0.3663        2.7659       0.2950        3.0715     +  0.0100  0.5918
      9       0.4288        2.3871       0.3325        2.9305     +  0

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [36,37,49,62,66,67,70] # 测试使用 
pre_save_path = "BETA_shallownet_mask_1s_0.9.pth"
# 加载预训练模型
n_channels = X_3s.shape[1]
n_samples = X_3s.shape[2]
n_classes = 40
shared_ratio = 0.9
alpha = 1
epsilon = 30
n_time_filters = 20
time_kernel = 25
n_space_filters = 15
pool_kernel = 8
pool_stride = 4


estimator = ShallowNet_Sparse(n_time_filters, time_kernel,n_space_filters,pool_kernel, pool_stride, n_channels, n_samples, n_classes, shared_ratio, alpha, epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = paradigm_3s.get_data(
        BETA_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
in:825
out:40
--------ssssss, /upload/liubingchuan/S31-S40.tar.gz
--------ssssss, /upload/liubingchuan/S31-S40.tar.gz
--------ssssss, /upload/liubingchuan/S41-S50.tar.gz
--------ssssss, /upload/liubingchuan/S61-S70.tar.gz
--------ssssss, /upload/liubingchuan/S61-S70.tar.gz
--------ssssss, /upload/liubingchuan/S61-S70.tar.gz
--------ssssss, /upload/liubingchuan/S61-S70.tar.gz
subject 36 的准确率: 50.0%
subject 37 的准确率: 67.5%
subject 49 的准确率: 48.1%
subject 62 的准确率: 65.0%
subject 66 的准确率: 50.0%
subject 67 的准确率: 52.5%
subject 70 的准确率: 46.2%
所有被试的准确率： {'36': 0.5, '37': 0.675, '49': 0.48125, '62': 0.65, '66': 0.5, '67': 0.525, '70': 0.4625}
所有被试的平均准确率: 54.2%
所有被试准确率（无名称）: 50.0,67.5,48.1,65.0,50.0,52.5,46.2


In [5]:
#**************************************************
# Benchmark数据集读取处理
#**************************************************
Bench_dataset = Wang2016()
subject_list = list(range(1, 21))  # 被试编号从1到20
for s in subject_list:
    Bench_dataset.data_path(subject=s, path="E:\\MetaBCI-master\\mne_data")  # 依次为每个被试设置路径
events = Bench_dataset.events.keys()
freq_list = [str(Bench_dataset.get_freq(event)) for event in events]  # 获得所有刺激的频率
freq_map = {i: freq for i, freq in enumerate(freq_list)}  # 标签到频率的映射

Bench_subjects = list(range(1, 21))  # 使用S1–S20被试进行训练
Bench_paradigm = SSVEP(
    channels=['POZ', 'PZ', 'PO3', 'PO5', 'PO4', 'PO6', 'O1', 'OZ', 'O2'],  # 选择电极通道
    intervals=[(0.14, 1.14)],  # 分析时间窗最长-0.5-5.5s ,0.0是刺激开始
    events=freq_list,  # 选择所有刺激频率
    srate=250  # 采样率
)

# add 5-90Hz bandpass filter in raw hook
# 对原始EEG信号做5-90Hz带通滤波。
def raw_hook(raw, caches):
    raw.filter(5, 90, l_trans_bandwidth=2, h_trans_bandwidth=5, phase='zero-double')
    caches['raw_stage'] = caches.get('raw_stage', -1) + 1
    return raw, caches

Bench_paradigm.register_raw_hook(raw_hook)



# 获取微调数据
X_Bench, y_Bench, meta_Bench = Bench_paradigm.get_data(
    Bench_dataset,
    subjects=Bench_subjects,
    return_concat=True,
    n_jobs=None,
    verbose=False
)


--------ssssss, /upload/yijun/S1.mat.7z
--------ssssss, /upload/yijun/S2.mat.7z
--------ssssss, /upload/yijun/S3.mat.7z
--------ssssss, /upload/yijun/S4.mat.7z
--------ssssss, /upload/yijun/S5.mat.7z
--------ssssss, /upload/yijun/S6.mat.7z
--------ssssss, /upload/yijun/S7.mat.7z
--------ssssss, /upload/yijun/S8.mat.7z
--------ssssss, /upload/yijun/S9.mat.7z
--------ssssss, /upload/yijun/S10.mat.7z
--------ssssss, /upload/yijun/S11.mat.7z
--------ssssss, /upload/yijun/S12.mat.7z
--------ssssss, /upload/yijun/S13.mat.7z
--------ssssss, /upload/yijun/S14.mat.7z
--------ssssss, /upload/yijun/S15.mat.7z
--------ssssss, /upload/yijun/S16.mat.7z
--------ssssss, /upload/yijun/S17.mat.7z
--------ssssss, /upload/yijun/S18.mat.7z
--------ssssss, /upload/yijun/S19.mat.7z
--------ssssss, /upload/yijun/S20.mat.7z
--------ssssss, /upload/yijun/S1.mat.7z
--------ssssss, /upload/yijun/S2.mat.7z
--------ssssss, /upload/yijun/S3.mat.7z
--------ssssss, /upload/yijun/S4.mat.7z
--------ssssss, /upload/yijun

In [6]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_eegnet.py # Bench数据集 S1~S20 学习率1e-4，F1=40,D=2,F2=80
# 所有折准确率: [0.87375, 0.88625, 0.87625, 0.87875, 0.85125, 0.8875]
# 平均准确率： 0.875625                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
shared_ratio = 0.9
alpha = 1
epsilon = 30
n_time_filters = 20
time_kernel = 25
n_space_filters = 15
pool_kernel = 8
pool_stride = 4

Bench_save_path = "Bench_shallownet_mask_1s_0.9.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = ShallowNet_Sparse(n_time_filters, time_kernel,n_space_filters,pool_kernel, pool_stride, n_channels, n_samples, n_classes, shared_ratio, alpha, epsilon)
    estimator.set_params(optimizer__lr=1e-2,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 计算验证集准确率
    val_preds = estimator.predict(X_Bench[new_valid_ind])
    val_acc = np.mean(val_preds == y_Bench[new_valid_ind])
    #print("当前折验证集准确率：", val_acc)
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
in:825
out:40
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0267        5.6730       0.0350        4.5234     +  0.0100  1.1628
      2       0.0720        5.0169       0.1275        3.8330     +  0.0100  0.9566
      3       0.2030        3.9570       0.3488        2.6521     +  0.0100  0.9390
      4       0.3897        2.8860       0.5112        2.0895     +  0.0100  0.8992
      5       0.4878        2.3603       0.5138        2.0223     +  0.0100  0.9187
      6       0.5367        2.0468       0.5487        1.8599     +  0.0100  0.9338
      7       0.5610        1.8606       0.5663        1.8421     +  0.0100  0.9695
      8       0.5920        1.7362       0.5787        1.7365     +  0.0100  0.9209
      9       0.6095        1.5870       0.5763        1.7431        0

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35]  # 测试使用 
pre_save_path = "Bench_shallownet_mask_1s_0.9.pth"
# 加载预训练模型

n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
shared_ratio = 0.9
alpha = 1
epsilon = 30
n_time_filters = 20
time_kernel = 25
n_space_filters = 15
pool_kernel = 8
pool_stride = 4

estimator = ShallowNet_Sparse(n_time_filters, time_kernel,n_space_filters,pool_kernel, pool_stride, n_channels, n_samples, n_classes, shared_ratio, alpha, epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
in:825
out:40
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 89.2%
subject 24 的准确率: 85.0%
subject 27 的准确率: 74.6%
subject 28 的准确率: 89.2%
subject 31 的准确率: 82.5%
subject 34 的准确率: 77.5%
subject 35 的准确率: 86.2%
所有被试的准确率： {'22': 0.8916666666666667, '24': 0.85, '27': 0.7458333333333333, '28': 0.8916666666666667, '31': 0.825, '34': 0.775, '35': 0.8625}
所有被试的平均准确率: 83.5%
所有被试准确率（无名称）: 89.2,85.0,74.6,89.2,82.5,77.5,86.2
